# Experiment 2

#### Model Setup

Run models in the following order, using their output labels as features for the next model:

1. Multilabel Linguistic Classifier
2. Multilabel Stereotype + Omission Document Classifier

Train the first model and then run it over the entire dataset.

***

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/experiment_input/`
    * Prediction Data: Data: under directory `../data/token_clf_data/model_output/experiment2/`
* Word Embeddings
    * Custom fastText (word2vec with subwords) embeddings of 100 dimensions trained on the CRC Archives catalog's descriptive metadata (harvested October 2020)
    
***

**Table of Contents**

[I.](#i) Stereotype + Omission Classifier
* [Preprocessing](#prep)
* [Training & Prediction](#tp)
* [Evaluation](#eval)

Load programming resources:

In [1]:
# For custom functions and variables
import utils, utils1, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For classification
import scipy
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix#, plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support

# For saving model
from joblib import dump,load

Define resources for the models:

In [2]:
### For 60-20-20 train-dev-test split of documents (metadata descriptions)
Path(config.experiment_input_path).mkdir(parents=True, exist_ok=True)    # For train, devtest, and blind test data
# Path(config.experiment1_output_path).mkdir(parents=True, exist_ok=True)  # For predictions
# Path(config.experiment1_agmt_path).mkdir(parents=True, exist_ok=True)    # For agreement metrics
# ----------------------
### For features from modified 5-fold CV
# predictions_dir = config.experiment2_path+"5fold/output/"
# Path(predictions_dir).mkdir(parents=True, exist_ok=True)    # For predictions
# agreement_dir = config.experiment2_path+"5fold/agreement/"
# Path(agreement_dir).mkdir(parents=True, exist_ok=True)      # For agreement metrics

predictions_dir = config.experiment2_path+"5fold/with_manual_labels/output/"              # For predictions with features as manual labels
Path(predictions_dir).mkdir(parents=True, exist_ok=True)  
agreement_dir = config.experiment2_path+"5fold/with_manual_labels/agreement/"             # For agreement metrics with features as manual labels
Path(agreement_dir).mkdir(parents=True, exist_ok=True)

In [3]:
# Model 1:
ling_label_subset = ["B-Generalization", "I-Generalization", "B-Gendered-Role", "I-Gendered-Role", "B-Gendered-Pronoun", "I-Gendered-Pronoun"]
# Model 3:
so_label_subset = ["B-Stereotype", "I-Stereotype", "B-Omission", "I-Omission"]

In [4]:
ling_label_tags = {
    "Gendered-Pronoun": ["B-Gendered-Pronoun", "I-Gendered-Pronoun"], "Gendered-Role": ["B-Gendered-Role", "I-Gendered-Role"],"Generalization": ["B-Generalization", "I-Generalization"]
    }
so_label_tags = {
    "Stereotype": ["B-Stereotype", "I-Stereotype"], "Omission": ["B-Omission", "I-Omission"]
             }

In [5]:
d = 100               # dimensions of word embeddings (should match utils1.py) for file names
target_labels = "so"  # for file names

<a id="i"></a>
## I. Stereotype + Omission Classifier
<a id="prep"></a>
### Preprocessing

Load the document classification model's input data:

In [ ]:
df = pd.read_csv(config.tokc_path+"experiment_input/document_5fold.csv", index_col=0)
df_exp = utils.getColumnValuesAsLists(df, "label")
df_exp = df_exp.drop(columns=["subset"])
df_exp.head()

,description_id,start_offset,end_offset,field,description,label,fold
0,4699,1853,2066,Biographical / Historical,"Labelled Apparently some chapters, amounting t...",[Omission],split3
1,8942,384,540,Biographical / Historical,James Aikman of Perth signed his name to a vol...,[],split2
2,5440,5692,5850,Biographical / Historical,This piece was published in 'Milk Production i...,[],split0
3,3474,3608,8549,Biographical / Historical,Margaret Winifred Bartholomew was born on 21 A...,"[Omission, Stereotype]",split0
4,4769,2378,2576,Biographical / Historical,Blacker and Thomson became close friends throu...,[Omission],split3


Load the Linguistic labels as features and associate description IDs to the data, creating one row per description ID:

In [ ]:
a="rf"
feature_col = "predicted_label"  # "predicted_tag"
df_features = pd.read_csv(config.experiment1_path+"5fold/output/cc-{a}_linglabels_baseline_fastText{d}_strict_evaluation.csv".format(a=a,d=d), usecols=["sentence_id", "token_id", feature_col])
df_features = df_features.fillna("O")
feature_col1 = "ling_predicted"  # "ling_pred"
df_features = df_features.rename(columns={feature_col:feature_col1})
df_features = utils.implodeDataFrame(df_features, ["sentence_id", "token_id"]).reset_index()
df_features.head()

,sentence_id,token_id,ling_predicted
0,0,0,[O]
1,0,1,[O]
2,0,2,[O]
3,1,3,[O]
4,1,4,[O]


In [8]:
df_desc = pd.read_csv(config.agg_path+"descs_sents_tokens_anns.csv", usecols=["description_id", "sentence_id", "token_id"])
df_desc = df_desc.set_index("description_id")
df_desc = utils1.getColumnValuesAsLists(df_desc, "sentence_id")
df_desc = utils1.getColumnValuesAsLists(df_desc, "token_id")
df_desc_exploded = df_desc.explode(["sentence_id", "token_id"])
df_desc_exploded = df_desc_exploded.reset_index()
df_desc_exploded = df_desc_exploded.astype("int64")
# df_desc_exploded.head()
assert df_desc_exploded.shape[0] == len(df_desc_exploded.token_id.unique())

In [9]:
joined = df_features.join(df_desc_exploded.set_index(["sentence_id", "token_id"]), on=["sentence_id", "token_id"])
grouped = utils.implodeDataFrame(joined, ["description_id"]).reset_index()
grouped.head()

,description_id,sentence_id,token_id,ling_predicted
0,0,"[0, 0, 0]","[0, 1, 2]","[[O], [O], [O]]"
1,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]","[[O], [O], [O], [O], [O, O, O], [O, O, O], [O,..."
2,2,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."
3,3,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[109, 110, 111, 112, 113, 114, 115, 116, 117, ...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."
4,4,"[11, 11, 11]","[308, 309, 310]","[[O], [O], [O]]"


Flatten the lists of values in the column of Linguistic label predictions and remove duplicates from the lists:

In [10]:
ling = utils1.flattenFeatureCol(grouped, feature_col1)
# ling[:10]

In [11]:
grouped.insert(len(grouped.columns), "doc_"+feature_col1, ling)
# grouped.head()

Join the Linguistic feature column to the document classification model data:

In [12]:
features = grouped[["description_id", "doc_"+feature_col1]]
join_on = "description_id"
df = df_exp.join(features.set_index(join_on), on=join_on)
df = df.loc[~df.description.isna()]
df = df.sort_values(by="description_id")
df.head()

,description_id,start_offset,end_offset,field,description,label,fold,doc_ling_predicted
5163,1,17,76,Title,Papers of The Very Rev Prof James Whyte (1920-...,[Stereotype],split2,[]
11541,2,77,633,Scope and Contents,"Sermons and addresses, 1948-1996; lectures, 19...",[],split3,[]
277,3,634,1725,Biographical / Historical,Professor James Aitken White was a leading Sco...,[Stereotype],split4,[Gendered-Pronoun]
19301,5,17,60,Title,Papers of Rev Tom Allan (1916-1965),[],split1,[]
21645,6,61,560,Scope and Contents,"Sermons and addresses, 1947-1963; essays and l...",[],split3,[]


Replace `NaN` values in last column with an empty list:

In [13]:
doc_ling = list(df["doc_"+feature_col1])
new_doc_ling = [values if type(values)==list else [] for values in doc_ling]
# new_doc_ling[:5]  # Looks good
df["doc_"+feature_col1] = new_doc_ling
df.head()

,description_id,start_offset,end_offset,field,description,label,fold,doc_ling_predicted
5163,1,17,76,Title,Papers of The Very Rev Prof James Whyte (1920-...,[Stereotype],split2,[]
11541,2,77,633,Scope and Contents,"Sermons and addresses, 1948-1996; lectures, 19...",[],split3,[]
277,3,634,1725,Biographical / Historical,Professor James Aitken White was a leading Sco...,[Stereotype],split4,[Gendered-Pronoun]
19301,5,17,60,Title,Papers of Rev Tom Allan (1916-1965),[],split1,[]
21645,6,61,560,Scope and Contents,"Sermons and addresses, 1947-1963; essays and l...",[],split3,[]


Define the train (80% of the data) and test (20% of the data) splits:

In [14]:
split_col = "fold"
splits = df[split_col].unique()
splits.sort()
print(splits)
train0, test0 = list(splits[:4]), splits[4]
train1, test1 = list(splits[1:]), splits[0]
train2, test2 = list(splits[2:])+[splits[0]], splits[1]
train3, test3 = list(splits[3:])+list(splits[:2]), splits[2]
train4, test4 = [splits[4]]+list(splits[:3]), splits[3]
runs = [(train0, test0), (train1, test1), (train2, test2), (train3, test3), (train4, test4)]
for run in runs:
    print(run)

['split0' 'split1' 'split2' 'split3' 'split4']
(['split0', 'split1', 'split2', 'split3'], 'split4')
(['split1', 'split2', 'split3', 'split4'], 'split0')
(['split2', 'split3', 'split4', 'split0'], 'split1')
(['split3', 'split4', 'split0', 'split1'], 'split2')
(['split4', 'split0', 'split1', 'split2'], 'split3')


In [15]:
def binarizeMultilabelTrainColumn(df_col):
    mlb = MultiLabelBinarizer()
    binarized = mlb.fit_transform(df_col)
    return mlb, binarized

def binarizeMultilabelDevColumn(mlb, df_col):
    binarized = mlb.transform(df_col)
    return binarized

<a id="tp"></a>
### Train & Predict

In [16]:
a = "sgd-svm"

In [17]:
pred_df = pd.DataFrame()
target_col = "label"
feat_col = "doc_ling_predicted"  #"doc_ling_expected"
for run in runs:
    # Get the train (80%) and test (20%) subsets of data
    train_splits, test_split = run[0], run[1]
    print("Training on:", train_splits)
    train_df = df.loc[df[split_col].isin(train_splits)]
    dev_df = df.loc[df[split_col] == test_split]
    
    # Binarize the features
    mlb_feat, train_feat = binarizeMultilabelTrainColumn(train_df[feat_col])
    dev_feat = binarizeMultilabelDevColumn(mlb_feat, dev_df[feat_col])
    
    # Vectorize the documents (descriptions)
    cvectorizer = CountVectorizer()
    tfidf = TfidfTransformer()
    train_docs = cvectorizer.fit_transform(train_df["description"])
    dev_docs = cvectorizer.transform(dev_df["description"])
    train_docs = tfidf.fit_transform(train_docs)
    dev_docs = tfidf.transform(dev_docs)
    
    # Concatenate the features and documents
    X_train = scipy.sparse.hstack([train_docs, train_feat])
    X_dev = scipy.sparse.hstack([dev_docs, dev_feat])
    
    # Binarize targets
    mlb_target, y_train = binarizeMultilabelTrainColumn(train_df["label"])
    y_dev = binarizeMultilabelDevColumn(mlb_target, dev_df["label"])

    # Train a classification model
    clf = OneVsRestClassifier(SGDClassifier(loss="hinge"))  # Support Vector Machines loss function
    clf.fit(X_train, y_train)
    
    # Predict with the trained model
    print("Predicting on:", test_split)
    predictions = clf.predict(X_dev)
    pred_labels = mlb_target.inverse_transform(predictions)    
    if pred_df.shape[0] > 0:
        next_pred_df = dev_df.copy()
        next_pred_df.insert(len(next_pred_df.columns), "{}_label".format(a), pred_labels)
        pred_df = pd.concat([pred_df, next_pred_df])
    else:
        pred_df = dev_df.copy()
        pred_df.insert(len(pred_df.columns), "{}_label".format(a), pred_labels)

print("Modified 5-fold cross-validation complete!")
print(pred_df.shape)

Training on: ['split0', 'split1', 'split2', 'split3']
Predicting on: split4
Training on: ['split1', 'split2', 'split3', 'split4']
Predicting on: split0
Training on: ['split2', 'split3', 'split4', 'split0']
Predicting on: split1
Training on: ['split3', 'split4', 'split0', 'split1']
Predicting on: split2
Training on: ['split4', 'split0', 'split1', 'split2']
Predicting on: split3
Modified 5-fold cross-validation complete!
(27312, 9)


In [18]:
pred_df = pred_df.rename(columns={"label":"manual_label"})
pred_df.head()

,description_id,start_offset,end_offset,field,description,manual_label,fold,doc_ling_predicted,sgd-svm_label
277,3,634,1725,Biographical / Historical,Professor James Aitken White was a leading Sco...,[Stereotype],split4,[Gendered-Pronoun],"(Stereotype,)"
20188,10,77,417,Scope and Contents,"Academic papers, 1942-1986; publications and a...",[],split4,[],"(,)"
6748,19,268,294,Title,Commonplace Book 2,[],split4,[],"(,)"
24617,28,540,556,Title,Notebook,[],split4,[],"(,)"
6873,30,581,609,Title,Poems and quotations,[],split4,[],"(,)"


Save the predictions data:

In [19]:
pred_df.to_csv(predictions_dir+"aggregated_final_validate_predictions_docclf_{a}_{t}.csv".format(a=a, t=target_labels))

Save the model:

In [20]:
model_dir = "models/experiment2/"
Path(model_dir).mkdir(parents=True, exist_ok=True)
filename = model_dir+"sgd-svm_F-tfidf_T-so.joblib"
dump(clf, filename)

['models/experiment2/sgd-svm_F-tfidf_T-so.joblib']

<a id="eval"></a>
### Evaluate

Calculate performance metrics for the Stochastic Gradient Descent classifier

In [21]:
classes = clf.classes_  #doc_clf.classes_
print(classes)
original_classes = mlb_target.classes_
print(original_classes)
label_dict = dict(zip(original_classes, classes))

[0 1 2]
['' 'Omission' 'Stereotype']


Create a [confusion matrix](https://scikit-learn.org/stable/modules/model_evaluation.html#multilabel-confusion-matrix) of the results, where, for class *i*:
* Count of true negatives (TN) is at position *i*,0,0
* Count of false negatives (FN) is at position *i*,1,0
* Count of true positives (FP) is at position *i*,1,1
* Count of false positives (PF) is at position *i*,0,1

In [22]:
y_dev = binarizeMultilabelDevColumn(mlb_target, pred_df["manual_label"])
predictions = mlb_target.transform(pred_df["{}_label".format(a)])
assert len(y_dev) == len(predictions)

In [23]:
matrix = multilabel_confusion_matrix(y_dev, predictions, labels=classes)

In [24]:
scores = utils.getPerformanceMetrics(y_dev, predictions, matrix, classes, original_classes, label_dict)
scores = scores.tail(2) # Remove row for 'None'
scores = scores.drop(columns="true_neg")  # Not accurate because considers 'None' a class
scores["labels"] = original_classes[1:]
scores

,labels,false_neg,true_pos,false_pos,precision,recall,f_1
1,Omission,1717,2315,450,0.837251,0.574157,0.681183
2,Stereotype,394,1207,76,0.940764,0.753904,0.837032


Save the performance results:

In [25]:
scores.to_csv(agreement_dir+"docclf_{a}_{t}_baseline_performance.csv".format(a=a, t=target_labels))